{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Complete Multi-Horizon Glucose Prediction Pipeline\n",
    "\n",
    "This notebook demonstrates the complete pipeline from data loading to model evaluation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "sys.path.append('../src')\n",
    "\n",
    "import numpy as np\n",
    "import tensorflow as tf\n",
    "from sklearn.model_selection import train_test_split\n",
    "\n",
    "# Import our modules\n",
    "from glucose_prediction.data.preprocessing import load_and_preprocess_data\n",
    "from glucose_prediction.data.sequence_generator import create_sequences\n",
    "from glucose_prediction.models.multi_horizon_model import train_and_evaluate_multi_horizon_model\n",
    "from glucose_prediction.evaluation.clarke_error_grid import evaluate_glucose_predictions_with_clarke\n",
    "from glucose_prediction.visualization.plotting import (\n",
    "    plot_training_history, plot_original_scale_predictions\n",
    ")\n",
    "from glucose_prediction.utils.config import ExperimentConfig\n",
    "\n",
    "# Set random seeds\n",
    "np.random.seed(42)\n",
    "tf.random.set_seed(42)\n",
    "\n",
    "print(\"All modules imported successfully!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configuration\n",
    "config = ExperimentConfig()\n",
    "\n",
    "# Update this path to your actual data file\n",
    "DATA_PATH = \"../data/raw/your_data.csv\"  # Replace with actual path\n",
    "\n",
    "print(f\"Configuration: {config.model.__dict__}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 1: Load and preprocess data\n",
    "print(\"Loading and preprocessing data...\")\n",
    "df, df_scaled, scaler_x, scaler_y = load_and_preprocess_data(\n",
    "    DATA_PATH, \n",
    "    config.model.sequence_length,\n",
    "    config.data.features\n",
    ")\n",
    "print(f\"Data loaded: {len(df)} records\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 2: Create sequences\n",
    "print(\"Creating sequences...\")\n",
    "X, y_60, y_120 = create_sequences(\n",
    "    df_scaled, \n",
    "    config.model.sequence_length,\n",
    "    config.model.prediction_horizon_60,\n",
    "    config.model.prediction_horizon_120\n",
    ")\n",
    "print(f\"Sequences created: X={X.shape}, y_60={y_60.shape}, y_120={y_120.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 3: Train model\n",
    "print(\"Training multi-horizon model...\")\n",
    "model, history, eval_data, results = train_and_evaluate_multi_horizon_model(\n",
    "    X, y_60, y_120, \n",
    "    config.model.sequence_length, \n",
    "    X.shape[2], \n",
    "    scaler_y,\n",
    "    epochs=config.model.epochs,\n",
    "    batch_size=config.model.batch_size,\n",
    "    validation_split=config.data.validation_split,\n",
    "    patience=config.model.patience\n",
    ")\n",
    "print(\"Training completed!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 4: Visualize results\n",
    "print(\"Generating visualizations...\")\n",
    "plot_training_history(history)\n",
    "plot_original_scale_predictions(eval_data, scaler_y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 5: Clinical evaluation\n",
    "print(\"Performing Clarke Error Grid Analysis...\")\n",
    "clarke_results = evaluate_glucose_predictions_with_clarke(eval_data, scaler_y)\n",
    "\n",
    "# Print summary\n",
    "print(\"\\nFinal Results Summary:\")\n",
    "print(\"====================\")\n",
    "print(f\"60-min R²: {results['60min']['metrics']['r2']:.4f}\")\n",
    "print(f\"120-min R²: {results['120min']['metrics']['r2']:.4f}\")\n",
    "print(f\"60-min Clinical Acceptability: {clarke_results['60min']['clinical_acceptability']:.1f}%\")\n",
    "print(f\"120-min Clinical Acceptability: {clarke_results['120min']['clinical_acceptability']:.1f}%\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}